# Introduction - Customer Churn Prediction notebook
In this notebook, we illustrate how you can train a model for Churn Prediction using scikit learn. After training the model, you step through the instructions to deploy the model using Watson Machine Learning.

This notebook is a variation of the original notebook reference in this github repo: https://github.com/elenalowery/cpd4_demo/blob/master/assets/jupyterlab/Predict_Customer_Churn_CPD4.ipynb


In [3]:
# Install required Python modules
!pip install sklearn-pandas > /dev/null


## Step 1: Review Use Case
The analytics use case implemented in this notebook is telco churn prediction. It is a simple use case which illustrates typical process for model development and deployment using Cloud Pak for Data.

In [4]:
import subprocess
CURRENT_BRANCH = subprocess.run(['git', 'rev-parse', '--abbrev-ref', 'HEAD'], stdout=subprocess.PIPE)\
    .stdout.strip().decode("utf-8")

if CURRENT_BRANCH in ['prd','uat']:
    CURRENT_ENV=CURRENT_BRANCH
else:
    CURRENT_ENV='dev'
    
print('Current branch     : {}'.format(CURRENT_BRANCH))
print('Current environment: {}'.format(CURRENT_ENV))

Current branch     : optimize-churn-model
Current environment: dev


In [5]:
import pandas as pd
customer_data_df=pd.read_csv('/userfs/assets/data_asset/CUSTOMER_DATA_ready-'+CURRENT_ENV+'.csv')
customer_data_df.head(10)

,ID,LONGDISTANCE,INTERNATIONAL,LOCAL,DROPPED,PAYMETHOD,LOCALBILLTYPE,LONGDISTANCEBILLTYPE,USAGE,RATEPLAN,...,CREDITCARD,DOB,ADDRESS_1,CITY,STATE,ZIP,ZIP4,LONGITUDE,LATITUDE,CHURN
0,1,23,0,206,0,CC,Budget,Intnl_discount,229,3,...,1.814139e+15,32dad3590f2243b8709201348e1ae897,159 HUTTON ST BSMT A,ABSECON,NJ,8201,0,NaN,NaN,T
1,1004,28,0,60,0,Auto,FreeLocal,Standard,89,4,...,6.494422e+15,c643e317495168f62085716c81ec164d,1724 WHITEHAVEN,GLYNDON,MN,56547,0,NaN,NaN,F
2,1005,24,0,5,0,CH,Budget,Standard,29,4,...,3.218720e+15,80c40ce517ca57e0919e238e0e29e75c,95 W 25TH ST APT 1,WAPPINGERS FALLS,NY,12590,1723,NaN,NaN,F
3,1006,28,0,97,0,CC,FreeLocal,Standard,125,1,...,3.016220e+15,df7b078f544b61f867ad0dc1fa51c046,66 KULLA DR,RICHLAND,NE,68601,0,-97.377539,41.441233,T
4,1008,0,0,4,2,CC,Budget,Standard,4,2,...,7.070216e+15,273a525adc7bb0bd49252e47dab190e9,5621 MCCARTY RD,EVERETT,WA,98205,0,NaN,NaN,F
5,1009,29,0,9,0,CC,Budget,Intnl_discount,38,2,...,4.919386e+15,efb18ce1ef44f169687df57e9b9fdf53,2000 CALLE 4,CAROLINA,PR,979,0,NaN,NaN,F
6,1010,13,0,40,0,CC,Budget,Standard,53,4,...,9.402648e+15,227f74a0e2d7b254a9c73ec61528ee94,3801 YOSEMITE BLVD STE F,HOUSTON,TX,77024,7776,NaN,NaN,F
7,1016,16,0,114,0,CH,Budget,Standard,130,1,...,8.522563e+15,92e4302092a290acd3bc1fb75ada5267,843 EUCLID ST APT 101S,KIRKLAND,WA,98034,0,-122.209175,47.709619,T
8,1017,7,0,6,0,CC,Budget,Standard,13,3,...,2.981966e+15,32bd821d9a01040a89f9a7d3766017ce,3801 MAC CV,NEW YORK,NY,10019,0,-73.990852,40.768196,F
9,1018,21,0,87,0,CC,Budget,Standard,108,1,...,3.074091e+15,e78d37c276f03bdfa0eef28dc18f9c3a,390 W BROADWAY ST,BUTLER,NJ,7405,0,NaN,NaN,F


In [6]:
# COPY the dataFrame into a new dataFrame called *data*
data=customer_data_df.copy()

In [7]:
# List all the columns
print(data.columns)

Index(['ID', 'LONGDISTANCE', 'INTERNATIONAL', 'LOCAL', 'DROPPED', 'PAYMETHOD',
       'LOCALBILLTYPE', 'LONGDISTANCEBILLTYPE', 'USAGE', 'RATEPLAN', 'GENDER',
       'STATUS', 'CHILDREN', 'ESTINCOME', 'CAROWNER', 'AGE', 'CREDITCARD',
       'DOB', 'ADDRESS_1', 'CITY', 'STATE', 'ZIP', 'ZIP4', 'LONGITUDE',
       'LATITUDE', 'CHURN'],
      dtype='object')


In [8]:
# Keep only the columns that are relevant for churn prediction
data = data[['LONGDISTANCE', 'INTERNATIONAL', 'LOCAL', 'DROPPED', 'PAYMETHOD', 'LOCALBILLTYPE', 'LONGDISTANCEBILLTYPE', 'USAGE', 'RATEPLAN', 'GENDER','STATUS', 'CHILDREN', 'ESTINCOME', 'CAROWNER', 'AGE', 'CHURN']]
data.head()


,LONGDISTANCE,INTERNATIONAL,LOCAL,DROPPED,PAYMETHOD,LOCALBILLTYPE,LONGDISTANCEBILLTYPE,USAGE,RATEPLAN,GENDER,STATUS,CHILDREN,ESTINCOME,CAROWNER,AGE,CHURN
0,23,0,206,0,CC,Budget,Intnl_discount,229,3,F,S,1,38000.00,N,24.393333,T
1,28,0,60,0,Auto,FreeLocal,Standard,89,4,F,M,1,8073.11,N,46.000000,F
2,24,0,5,0,CH,Budget,Standard,29,4,M,M,0,95448.60,Y,53.680000,F
3,28,0,97,0,CC,FreeLocal,Standard,125,1,M,S,1,24141.50,Y,17.006667,T
4,0,0,4,2,CC,Budget,Standard,4,2,M,S,1,31952.00,N,34.266667,F


## Step 2: Try the Random Forest model

In [9]:
import pandas as pd
import sklearn
pd.options.display.max_columns = 999

import warnings
warnings.filterwarnings('ignore')

from scipy.stats import chi2_contingency,ttest_ind
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.model_selection import train_test_split

from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, accuracy_score, roc_curve, roc_auc_score

import numpy as np

import urllib3, requests, json

In [10]:
#convert CHURN to 1/0
le = LabelEncoder()
data.loc[:,'CHURN']= le.fit_transform(data.loc[:,'CHURN'])
data.head()

,LONGDISTANCE,INTERNATIONAL,LOCAL,DROPPED,PAYMETHOD,LOCALBILLTYPE,LONGDISTANCEBILLTYPE,USAGE,RATEPLAN,GENDER,STATUS,CHILDREN,ESTINCOME,CAROWNER,AGE,CHURN
0,23,0,206,0,CC,Budget,Intnl_discount,229,3,F,S,1,38000.00,N,24.393333,1
1,28,0,60,0,Auto,FreeLocal,Standard,89,4,F,M,1,8073.11,N,46.000000,0
2,24,0,5,0,CH,Budget,Standard,29,4,M,M,0,95448.60,Y,53.680000,0
3,28,0,97,0,CC,FreeLocal,Standard,125,1,M,S,1,24141.50,Y,17.006667,1
4,0,0,4,2,CC,Budget,Standard,4,2,M,S,1,31952.00,N,34.266667,0


In [11]:
# define the label and features
y = np.float32(data.CHURN)
x = data.drop(['CHURN','RATEPLAN','GENDER','ESTINCOME','STATUS','AGE','USAGE'], axis = 1)

In [12]:
x.columns

Index(['LONGDISTANCE', 'INTERNATIONAL', 'LOCAL', 'DROPPED', 'PAYMETHOD',
       'LOCALBILLTYPE', 'LONGDISTANCEBILLTYPE', 'CHILDREN', 'CAROWNER'],
      dtype='object')

In [13]:
# Apply the LabelEncoder to encode the input features in numeric form where applicable
from sklearn_pandas import DataFrameMapper

'''
mapper = DataFrameMapper(
    [('GENDER', LabelEncoder()),
     ('STATUS', LabelEncoder()),
     ('CHILDREN', None),
     ('ESTINCOME',None),
     ('CAROWNER', LabelEncoder()),
     ('AGE',None),
     ('LONGDISTANCE',None),
     ('INTERNATIONAL',None),
     ('LOCAL',None),
     ('DROPPED',None),
     ('PAYMETHOD',LabelEncoder()),
     ('LOCALBILLTYPE',LabelEncoder()),
     ('LONGDISTANCEBILLTYPE',LabelEncoder()),
     ('USAGE',None),
     ('RATEPLAN',None)
    ]
)
'''

mapper = DataFrameMapper(
    [
     ('CHILDREN', None),
     ('CAROWNER', LabelEncoder()),
     ('LONGDISTANCE',None),
     ('INTERNATIONAL',None),
     ('LOCAL',None),
     ('DROPPED',None),
     ('PAYMETHOD',LabelEncoder()),
     ('LOCALBILLTYPE',LabelEncoder()),
     ('LONGDISTANCEBILLTYPE',LabelEncoder())
    ]
)

In [14]:
# split the data to training and testing set
X_train, X_test, y_train, y_test = train_test_split(x, y, 
    test_size=0.2, 
    random_state=42, stratify=y)

In [15]:
# fit the model

import sklearn.pipeline
from sklearn.preprocessing import OneHotEncoder

random_forest = RandomForestClassifier()
steps = [('mapper', mapper),('RandonForestClassifier', random_forest)]
pipeline = sklearn.pipeline.Pipeline(steps)
model=pipeline.fit( X_train, y_train )
model

Pipeline(steps=[('mapper',
                 DataFrameMapper(drop_cols=[],
                                 features=[('CHILDREN', None),
                                           ('CAROWNER', LabelEncoder()),
                                           ('LONGDISTANCE', None),
                                           ('INTERNATIONAL', None),
                                           ('LOCAL', None), ('DROPPED', None),
                                           ('PAYMETHOD', LabelEncoder()),
                                           ('LOCALBILLTYPE', LabelEncoder()),
                                           ('LONGDISTANCEBILLTYPE',
                                            LabelEncoder())])),
                ('RandonForestClassifier', RandomForestClassifier())])

In [16]:
### call pipeline.predict() on your X_test data to make a set of test predictions
y_prediction = pipeline.predict( X_test )

### test your predictions using sklearn.classification_report()

report = sklearn.metrics.classification_report( y_test, y_prediction )
### and print the report
print(report)

              precision    recall  f1-score   support

         0.0       0.98      0.92      0.95       168
         1.0       0.90      0.97      0.93       115

    accuracy                           0.94       283
   macro avg       0.94      0.95      0.94       283
weighted avg       0.95      0.94      0.94       283



### Evaluate

Accuracy of the trained model is very good so we can now decide to deploy this model to be used by the applications.

## Step 3 - WML Deployment
In the next set of cells, we deploy the trained model using Watson Machine Learning into the space associated with the current environment.

In [17]:
import os
cpdtoken=os.environ['USER_ACCESS_TOKEN']
wml_credentials = {
"token": cpdtoken,
"instance_id" : "openshift",
"url": os.environ['RUNTIME_ENV_APSX_URL'],
"version": "4.0"
}

from ibm_watson_machine_learning import APIClient
client = APIClient(wml_credentials)

In [18]:
# Associate WML client with current project
project_id = os.environ['PROJECT_ID']
client.set.default_project(project_id)

'SUCCESS'

In [19]:
# Specify a name for the space being created, the saved model and the model deployment
space_name = 'churn-' + CURRENT_ENV
model_name = 'churn_pipeline'
deployment_name = 'churn_pipeline_deployment'

use_existing_space=True

In [20]:
from ibm_watson_machine_learning import APIClient
import os
import time

token = os.environ['USER_ACCESS_TOKEN']

wml_credentials = {
   "token": token,
   "instance_id" : "openshift",
   "url": os.environ['RUNTIME_ENV_APSX_URL'],
   "version": "4.0"
}

client = APIClient(wml_credentials)

### Create the deployment space if it doesn't exist yet

In [21]:
space_uid = ""
for space in client.spaces.get_details()['resources']:

    if space['entity']['name'] ==space_name:
        print("Deployment space with name",space_name,"already exists . .")
        space_uid=space['metadata']['id']
        client.set.default_space(space_uid)
        if(use_existing_space==False):

            for deployment in client.deployments.get_details()['resources']:
                print("Deleting deployment",deployment['entity']['name'], "in the space",)
                deployment_id=deployment['metadata']['id']
                client.deployments.delete(deployment_id)
            print("Deleting Space ",space_name,)
            client.spaces.delete(space_uid)
            time.sleep(5)
        else:
            print("Using the existing space")

if (space_uid == "" or use_existing_space == False):
    print("\nCreating a new deployment space -",space_name)
    # create the space and set it as default
    space_meta_data = {
        client.spaces.ConfigurationMetaNames.NAME : space_name

        }

    stored_space_details = client.spaces.store(space_meta_data)

    space_uid = stored_space_details['metadata']['id']

    client.set.default_space(space_uid)


Creating a new deployment space - churn-dev
Space has been created. However some background setup activities might still be on-going. Check for 'status' field in the response. It has to show 'active' before space can be used. If it's not 'active', you can monitor the state with a call to spaces.get_details(space_id). Alternatively, use background_mode=False when calling client.spaces.store().


In [22]:
client.software_specifications.list()

------------------------------  ------------------------------------  ----  ------------------  ------------------------------
NAME                            ID                                    TYPE  STATE               REPLACEMENT
default_py3.6                   0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base  unsupported         runtime-23.1-py3.10
runtime-22.1-r3.6               018ebea5-1d1f-5fec-b93e-5e2ab30e7f38  base  unsupported         runtime-23.1-r4.2
autoai-ts_rt23.1-py3.10         01ce9391-1a79-5a33-94fb-2e134337f314  base  supported
kernel-spark3.2-scala2.12       020d69ce-7ac1-5e68-ac1a-31189867356a  base  not_provided
tensorflow_rt23.1-py3.10-edt    067048c7-c771-5933-9aa0-4f9fbdc92da8  base  supported
pytorch-onnx_1.3-py3.7-edt      069ea134-3346-5748-b513-49120e15d288  base  not_provided
tensorflow_rt23.1-py3.10        079a91e0-245f-5269-8926-3c20b28f37dc  base  supported
scikit-learn_0.20-py3.6         09c5a1d0-9c1e-4473-a344-eb7b665ff687  base  unsupported         run

,NAME,ID,TYPE,STATE,REPLACEMENT
0,default_py3.6,0062b8c9-8b7d-44a0-a9b9-46c416adcbd9,base,unsupported,runtime-23.1-py3.10
1,runtime-22.1-r3.6,018ebea5-1d1f-5fec-b93e-5e2ab30e7f38,base,unsupported,runtime-23.1-r4.2
2,autoai-ts_rt23.1-py3.10,01ce9391-1a79-5a33-94fb-2e134337f314,base,supported,
3,kernel-spark3.2-scala2.12,020d69ce-7ac1-5e68-ac1a-31189867356a,base,not_provided,
4,tensorflow_rt23.1-py3.10-edt,067048c7-c771-5933-9aa0-4f9fbdc92da8,base,supported,
5,pytorch-onnx_1.3-py3.7-edt,069ea134-3346-5748-b513-49120e15d288,base,not_provided,
6,tensorflow_rt23.1-py3.10,079a91e0-245f-5269-8926-3c20b28f37dc,base,supported,
7,scikit-learn_0.20-py3.6,09c5a1d0-9c1e-4473-a344-eb7b665ff687,base,unsupported,runtime-23.1-py3.10
8,spark-mllib_3.0-scala_2.12,09f4cff0-90a7-5899-b9ed-1ef348aebdee,base,unsupported,
9,pytorch-onnx_rt22.1-py3.9,0b848dd4-e681-5599-be41-b5f6fccc6471,base,create-unsupported,pytorch-onnx_rt23.1-py3.10


In [24]:
software_spec_uid = client.software_specifications.get_uid_by_name('default_py3.7_opence')

metadata = {
    client.repository.ModelMetaNames.NAME: model_name,
    client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: software_spec_uid,
    client.repository.ModelMetaNames.TYPE: "scikit-learn_0.23"
}

stored_model_details = client.repository.store_model(pipeline,
                                               meta_props=metadata,
                                               training_data=X_train,
                                               training_target=y_train)

Traceback (most recent call last):
  File "/opt/conda/envs/Python-RT23.1-Premium/lib/python3.10/site-packages/ibm_watson_machine_learning/libs/repo/mlrepositoryclient/model_collection.py", line 245, in _save_scikit_pipeline_model
    model_artifact = self._create_pipeline_model(artifact, query_param)
  File "/opt/conda/envs/Python-RT23.1-Premium/lib/python3.10/site-packages/ibm_watson_machine_learning/libs/repo/mlrepositoryclient/model_collection.py", line 505, in _create_pipeline_model
    model_artifact = self._create_pipeline_model_v4_cloud(model_artifact, query_param)
  File "/opt/conda/envs/Python-RT23.1-Premium/lib/python3.10/site-packages/ibm_watson_machine_learning/libs/repo/mlrepositoryclient/model_collection.py", line 1018, in _create_pipeline_model_v4_cloud
    model_output = self.repository_api.ml_assets_model_creation_v4_cloud(model_input, query_param,headers)
  File "/opt/conda/envs/Python-RT23.1-Premium/lib/python3.10/site-packages/ibm_watson_machine_learning/libs/repo/s

WMLClientError: Publishing model failed.
Reason: (400)
Reason: Bad Request
HTTP response headers: HTTPHeaderDict({'Date': 'Wed, 17 Apr 2024 12:33:56 GMT', 'Content-Type': 'application/json', 'Content-Length': '287', 'Connection': 'keep-alive', 'Server': '---', 'Strict-Transport-Security': 'max-age=31536000; includeSubDomains'})
HTTP response body: {
  "trace": "bde7dflee57b9lg0441deu3xod2d7g02mb4x",
  "errors": [{
    "code": "invalid_request_entity",
    "message": "Invalid request entity: Unsupported model type 'scikit-learn_0.23'",
    "more_info": "https://cloud.ibm.com/apidocs/machine-learning"
  }],
  "status_code": "400"
}


In [86]:
# Confirm the model is stored in WML repository
client.repository.list_models()

------------------------------------  --------------  ------------------------  -----------------
ID                                    NAME            CREATED                   TYPE
68188f43-9418-4d97-90bd-ff7a9a4aec5f  churn_pipeline  2022-01-29T15:13:41.002Z  scikit-learn_0.23
c6be998a-f20a-4286-9849-bd8081b901bb  churn_pipeline  2022-01-29T15:12:18.002Z  scikit-learn_0.23
------------------------------------  --------------  ------------------------  -----------------


In [87]:
stored_model_details

{'entity': {'label_column': 'l1',
  'software_spec': {'id': 'c2057dd4-f42c-5f77-a02f-72bdbd3282c9',
   'name': 'default_py3.7_opence'},
  'training_data_references': [{'connection': {'access_key_id': 'not_applicable',
     'endpoint_url': 'not_applicable',
     'secret_access_key': 'not_applicable'},
    'id': '1',
    'location': {},
    'schema': {'fields': [{'name': 'LONGDISTANCE', 'type': 'int64'},
      {'name': 'INTERNATIONAL', 'type': 'int64'},
      {'name': 'LOCAL', 'type': 'int64'},
      {'name': 'DROPPED', 'type': 'int64'},
      {'name': 'PAYMETHOD', 'type': 'object'},
      {'name': 'LOCALBILLTYPE', 'type': 'object'},
      {'name': 'LONGDISTANCEBILLTYPE', 'type': 'object'},
      {'name': 'CHILDREN', 'type': 'int64'},
      {'name': 'CAROWNER', 'type': 'object'}],
     'id': '1',
     'type': 'DataFrame'},
    'type': 'fs'}],
  'type': 'scikit-learn_0.23'},
 'metadata': {'created_at': '2022-01-29T15:13:41.536Z',
  'id': '68188f43-9418-4d97-90bd-ff7a9a4aec5f',
  'modified

In [88]:
# Deploy the model
deploy_metadata = {
    client.deployments.ConfigurationMetaNames.NAME: deployment_name,
    client.deployments.ConfigurationMetaNames.ONLINE: {}
}

published_model_uid = client.repository.get_model_uid(stored_model_details)
created_deployment = client.deployments.create(published_model_uid, meta_props=deploy_metadata)




#######################################################################################

Synchronous deployment creation for uid: '68188f43-9418-4d97-90bd-ff7a9a4aec5f' started

#######################################################################################


initializing
Note: online_url is deprecated and will be removed in a future release. Use serving_urls instead.

ready


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='94a3099b-dc4b-4840-bc0b-d810a9934f83'
------------------------------------------------------------------------------------------------




In [89]:
deployment_uid = client.deployments.get_uid(created_deployment)
scoring_endpoint = client.deployments.get_scoring_href(created_deployment)
print(scoring_endpoint)

https://internal-nginx-svc:12443/ml/v4/deployments/94a3099b-dc4b-4840-bc0b-d810a9934f83/predictions


### Delete the old deployments of the previous model(s)

In [90]:
for deployment in client.deployments.get_details()['resources']:
    if (deployment['metadata']['name']==deployment_name) and (deployment['metadata']['id'] != deployment_uid):
        print('Deployment '+ deployment['metadata']['id'] + ' will be deleted')
        client.deployments.delete(deployment['metadata']['id'])

Deployment 7e6fb211-aa50-43cb-b92d-c5ba63150c2c will be deleted


### Delete previous models

In [91]:
for model in client.repository.get_model_details()['resources']:
    if (model['metadata']['name']==model_name) and (model['metadata']['id'] != stored_model_details['metadata']['id']):
        print('Model '+ model['metadata']['id'] + ' will be deleted')
        client.repository.delete(model['metadata']['id'])

Model c6be998a-f20a-4286-9849-bd8081b901bb will be deleted


### List remaining models and deployments

In [92]:
client.repository.list_models()
client.deployments.list()

------------------------------------  --------------  ------------------------  -----------------
ID                                    NAME            CREATED                   TYPE
68188f43-9418-4d97-90bd-ff7a9a4aec5f  churn_pipeline  2022-01-29T15:13:41.002Z  scikit-learn_0.23
------------------------------------  --------------  ------------------------  -----------------
------------------------------------  -------------------------  -----  ------------------------
GUID                                  NAME                       STATE  CREATED
94a3099b-dc4b-4840-bc0b-d810a9934f83  churn_pipeline_deployment  ready  2022-01-29T15:13:44.942Z
------------------------------------  -------------------------  -----  ------------------------


### Run a test score of the newly deployed model

In [93]:
# Score the model on a test dataset
scoring_payload = {
    "input_data": [{
        'fields': ['LONGDISTANCE', 'INTERNATIONAL', 'LOCAL', 'DROPPED', 'PAYMETHOD', 
                   'LOCALBILLTYPE', 'LONGDISTANCEBILLTYPE', 'CHILDREN', 'CAROWNER'],
        'values': [[28,0,60,0,"Auto","FreeLocal","Standard",1,"N"]]}]
}


In [94]:
predictions = client.deployments.score(deployment_uid, scoring_payload)
print(json.dumps(predictions, indent=2))

{
  "predictions": [
    {
      "fields": [
        "prediction",
        "probability"
      ],
      "values": [
        [
          0.0,
          [
            1.0,
            0.0
          ]
        ]
      ]
    }
  ]
}
